In [1]:
# 사용할 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 전 기본 세팅
#한글 깨짐
plt.rc("font", family="Malgun Gothic")

In [2]:
# 파일 불러오기
sample_submission = pd.read_csv('data/sample_submission.csv', index_col='no')
test = pd.read_csv('data/test.csv', index_col='no')
train = pd.read_csv('data/train.csv', index_col='no')

## 예측할 값 : 개인 소득
- income : 50K 초과는 1, 50K 이하는 0 (소득)

### 컬럼 의미
- age - continuous.
- workclass(일 유형) : Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt(일련의 관측 결과를 바탕으로 인구조사국이 부여하는 개인의 가중치): continuous.
- education(교육수준) : Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num(교육수준 번호) : continuous.
- marital-status(결혼 상태) : Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation(직업) : Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship(가족관계) : Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race(인종) : White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex(성별) : Female, Male.
- capital-gain(자본 이익) : continuous.
- capital-loss(자본 손실) : continuous.
- hours-per-week(주당 근무 시간) : continuous.
- native-country(국적) : United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

### value값 띄어쓰기 제거

In [3]:
# 띄어쓰기 제거함
for i in train.columns : 
    if train[f'{i}'].dtype == 'object' :
        train[f'{i}'] = train[f'{i}'].apply(lambda x: x.replace(' ', '')) # 제거하는 함수
    else :
        train[f'{i}'] = train[f'{i}']
        
for i in test.columns : 
    if test[f'{i}'].dtype == 'object' :
        test[f'{i}'] = test[f'{i}'].apply(lambda x: x.replace(' ', '')) # 제거하는 함수
    else :
        test[f'{i}'] = test[f'{i}']

In [4]:
title=['Private', 'Self-emp-not-inc', 'Local-gov', '?',
       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']
cover_title=['Private', 'Self-emp-not-inc', 'Local-gov', 'Never-worked',
       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']
title_dict = dict(zip(title, cover_title))
title_dict
train['workclass'] = train['workclass'].map(title_dict)
test['workclass'] =test['workclass'].map(title_dict)

In [5]:
title=['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       '?', 'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Portugal', 'Columbia', 'Hungary', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
cover_title=['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       'United-States', 'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Portugal', 'Columbia', 'Hungary', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
title_dict = dict(zip(title, cover_title))
title_dict
train['native-country'] = train['native-country'].map(title_dict)

test['native-country'] = test['native-country'].map(title_dict)

In [6]:
title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',
       'Tech-support', 'Transport-moving', 'Farming-fishing',
       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',
       'Protective-serv', 'Exec-managerial', '?', 'Craft-repair', 'Sales',
       'Armed-Forces']
cover_title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',
       'Tech-support', 'Transport-moving', 'Farming-fishing',
       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',
       'Protective-serv', 'Exec-managerial', 'Prof-specialty', 'Craft-repair', 'Sales',
       'Armed-Forces']
title_dict = dict(zip(title, cover_title))
title_dict
train['occupation'] = train['occupation'].map(title_dict)
test['occupation'] = test['occupation'].map(title_dict)

In [7]:
train = train.drop('education',axis=1)
test = test.drop('education',axis=1)

In [8]:
marital_status = ['Divorced', 'Never-married', 'Married-civ-spouse', 'Separated',
       'Married-spouse-absent', 'Widowed', 'Married-AF-spouse']
conver_marital_status = ['not-married', 'not-married', 'married', 'not-married', 'not-married', 'not-married','married']
marital_status_dict=dict(zip(marital_status, conver_marital_status))

train['marital-status']=train['marital-status'].map(marital_status_dict)
test['marital-status']=test['marital-status'].map(marital_status_dict)

In [9]:
train['relationship'].unique()
title=['Not-in-family', 'Husband', 'Unmarried', 'Other-relative',
       'Own-child', 'Wife']
cover_title=['Not-in-family', 'Married', 'Unmarried', 'Other-relative',
       'Own-child', 'Married']
title_dict = dict(zip(title, cover_title))
train['relationship'] = train['relationship'].map(title_dict)

title=['Own-child', 'Wife', 'Husband', 'Not-in-family', 'Unmarried',
       'Other-relative']
cover_title=['Own-child', 'Married', 'Married', 'Not-in-family', 'Unmarried',
       'Other-relative']
title_dict = dict(zip(title, cover_title))
test['relationship'] = test['relationship'].map(title_dict)

In [10]:
bins = [0,4,8,12,16] # 범위지정
labels = ['e1', 'e2', 'e3', 'e4'] # 라벨지정
train['education-num'] = pd.cut(train['education-num'], bins=bins, labels = labels)
test['education-num'] = pd.cut(test['education-num'], bins=bins, labels=labels)

In [11]:
"""#  사회 초년생 - age1 [17~30]
age1 = [i for i in range(17, 31)]
cover_age1 = ['young'] * (31-17)

#  중년층 - age2 [31~50]
age2 = [i for i in range(31, 51)]
cover_age2 = ['middle'] * (51-31)

# 노년층 - age3 [51 ~ 90]
age3 = [i for i in range(51, 91)]
cover_age3 = ['old'] * (91-51)

age_dict = dict(zip(age1 + age2 + age3, cover_age1 + cover_age2 + cover_age3))

train['age']=train['age'].map(age_dict)
test['age']=test['age'].map(age_dict)"""

"#  사회 초년생 - age1 [17~30]\nage1 = [i for i in range(17, 31)]\ncover_age1 = ['young'] * (31-17)\n\n#  중년층 - age2 [31~50]\nage2 = [i for i in range(31, 51)]\ncover_age2 = ['middle'] * (51-31)\n\n# 노년층 - age3 [51 ~ 90]\nage3 = [i for i in range(51, 91)]\ncover_age3 = ['old'] * (91-51)\n\nage_dict = dict(zip(age1 + age2 + age3, cover_age1 + cover_age2 + cover_age3))\n\ntrain['age']=train['age'].map(age_dict)\ntest['age']=test['age'].map(age_dict)"

In [12]:
bins = [i for i in range(0, 101,10)]
labels = ['age' + str(i) for i in range(0, 100,10)] # 라벨지정
train['age'] = pd.cut(train['age'], bins=bins, labels = labels)
test['age'] = pd.cut(test['age'], bins=bins, labels=labels)

In [13]:
title=['Private', 'Self-emp-not-inc', 'Local-gov', 'State-gov',
       'Self-emp-inc', 'Federal-gov', 'Without-pay', 'Never-worked']
cover_title=['Private', 'Self-emp', 'gov', 'gov',
       'Self-emp', 'gov', 'beggar', 'beggar']
title_dict = dict(zip(title, cover_title))
train['workclass'] = train['workclass'].map(title_dict)
test['workclass'] = test['workclass'].map(title_dict)

In [14]:
native_country = ['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Columbia', 'Hungary', 'Portugal', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
conver_native_country = ['United-States', 'North America', 'Asia', 'Europe', 'North America',
                         'North America', 'North America', 'North America', 'Asia', 'Europe',
                         'North America', 'Asia', 'Asia', 'North America', 'Europe',
                         'Asia', 'South America', 'North America', 'Asia', 'Europe',
                        'North America', 'Europe', 'Asia', 'Asia', 'North America',
                         'South America', 'Europe', 'Europe', 'South America', 'Asia',
                         'Europe', 'Asia', 'North America', 'Asia', 'Europe',
                         'Asia', 'South America', 'Europe', 'North America', 'Europe', 'Europe']

native_country_dict = dict(zip(native_country, conver_native_country))
native_country_dict
train['native-country']=train['native-country'].map(native_country_dict)
test['native-country']=test['native-country'].map(native_country_dict)

In [15]:
train = train.drop('fnlwgt',axis=1)

In [16]:
test= test.drop('fnlwgt',axis=1)

- 컬럼순서 바꾸기

In [17]:
train

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
no,,,,,,,,,,,,,
1,age20,Private,e2,not-married,Machine-op-inspct,Not-in-family,White,Male,0,0,40,United-States,0
2,age30,Private,e3,not-married,Other-service,Not-in-family,White,Female,0,1721,55,United-States,0
3,age30,Private,e4,not-married,Handlers-cleaners,Not-in-family,Asian-Pac-Islander,Female,0,0,50,North America,0
4,age60,Private,e3,not-married,Tech-support,Not-in-family,White,Female,0,0,40,United-States,0
5,age20,Private,e3,married,Transport-moving,Married,White,Male,0,0,70,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29301,age10,Private,e3,not-married,Sales,Own-child,Other,Male,0,0,40,Europe,0
29302,age20,Private,e3,not-married,Sales,Own-child,Black,Male,0,0,40,United-States,0
29303,age10,Self-emp,e2,not-married,Farming-fishing,Own-child,White,Male,0,0,40,United-States,0


In [18]:
# 문제/답 분리
X_train = train.loc[:,:'native-country']
y_train = train['income']

In [19]:
X_train

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
no,,,,,,,,,,,,
1,age20,Private,e2,not-married,Machine-op-inspct,Not-in-family,White,Male,0,0,40,United-States
2,age30,Private,e3,not-married,Other-service,Not-in-family,White,Female,0,1721,55,United-States
3,age30,Private,e4,not-married,Handlers-cleaners,Not-in-family,Asian-Pac-Islander,Female,0,0,50,North America
4,age60,Private,e3,not-married,Tech-support,Not-in-family,White,Female,0,0,40,United-States
5,age20,Private,e3,married,Transport-moving,Married,White,Male,0,0,70,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...
29301,age10,Private,e3,not-married,Sales,Own-child,Other,Male,0,0,40,Europe
29302,age20,Private,e3,not-married,Sales,Own-child,Black,Male,0,0,40,United-States
29303,age10,Self-emp,e2,not-married,Farming-fishing,Own-child,White,Male,0,0,40,United-States


In [20]:
y_train

no
1        0
2        0
3        0
4        0
5        1
        ..
29301    0
29302    0
29303    0
29304    0
29305    0
Name: income, Length: 29305, dtype: int64

In [21]:
X_train = pd.get_dummies(X_train) # 0과 1로만 이루어진 열을 생성
X_test = pd.get_dummies(test)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

- 앙상블

In [23]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

- 보팅

In [24]:
# 모델 생성
knn_model = KNeighborsClassifier(n_neighbors=5)
tree_model = DecisionTreeClassifier(max_depth=5)
forest_model = RandomForestClassifier(n_estimators=1000,
                                 max_depth=5,         
                                 min_samples_leaf=15,
                                 max_leaf_nodes=50)

In [25]:
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
mm_scaler.fit(X_train)
mm_transform_X_train = mm_scaler.transform(X_train)
mm_transform_X_train = pd.DataFrame(mm_transform_X_train, columns=X_train.columns)
mm_transform_X_test = mm_scaler.transform(X_test)
mm_transform_X_test = pd.DataFrame(mm_transform_X_test, columns=X_test.columns)

In [26]:
knn_model.fit(mm_transform_X_train,y_train)
tree_model.fit(X_train,y_train)
forest_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, max_leaf_nodes=50, min_samples_leaf=15,
                       n_estimators=1000)

- 부스팅

In [27]:
# AdaBoost모델생성
Ada_model = AdaBoostClassifier(n_estimators=100)

In [28]:
# 교차검증
result = cross_val_score(Ada_model,
                          X_train,
                          y_train,
                          cv = 5)
result.mean()

0.85865893192288

In [29]:
# 학습
Ada_model.fit(X_train,y_train)
# 예측
pre = Ada_model.predict(X_test)

In [30]:
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('ada_model6.csv', index = False)

In [31]:
# 모델 생성
rf_model = RandomForestClassifier(max_depth=20,
                                        max_leaf_nodes=80,
                                        min_samples_leaf=10,
                                        n_estimators=1500)

In [32]:
result = cross_val_score(rf_model,
                          X_train,
                          y_train,
                          cv = 5)

In [33]:
result.mean()

0.8544275720866746

In [34]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, max_leaf_nodes=80, min_samples_leaf=10,
                       n_estimators=1500)

In [35]:
pre = rf_model.predict(X_test)

In [36]:
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('rf_model6.csv', index = False)

In [37]:
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier()
result = cross_val_score(lgbm_model,
                         X_train,
                         y_train,
                         cv = 5)
print(result.mean())

lgbm_model.fit(X_train, y_train)
pre = lgbm_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('lgbm_model2.csv', index = False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5636, number of negative: 17808
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000704 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 340
[LightGBM] [Info] Number of data points in the train set: 23444, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5636, number of negative: 17808
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_

In [38]:
from catboost import CatBoostClassifier
cb_model = CatBoostClassifier(verbose = 200)
result = cross_val_score(cb_model,
                         X_train,
                         y_train,
                         cv = 5)
print(result.mean())
cb_model.fit(X_train, y_train)
pre = cb_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('cb_model2.csv', index = False)

Learning rate set to 0.039622
0:	learn: 0.6535233	total: 143ms	remaining: 2m 22s
200:	learn: 0.2866496	total: 1.04s	remaining: 4.14s
400:	learn: 0.2692787	total: 1.96s	remaining: 2.93s
600:	learn: 0.2596979	total: 2.83s	remaining: 1.88s
800:	learn: 0.2531792	total: 3.65s	remaining: 906ms
999:	learn: 0.2480846	total: 4.52s	remaining: 0us
Learning rate set to 0.039622
0:	learn: 0.6547496	total: 5.36ms	remaining: 5.35s
200:	learn: 0.2922605	total: 843ms	remaining: 3.35s
400:	learn: 0.2746913	total: 1.69s	remaining: 2.53s
600:	learn: 0.2653492	total: 2.54s	remaining: 1.69s
800:	learn: 0.2590310	total: 3.4s	remaining: 846ms
999:	learn: 0.2540211	total: 4.27s	remaining: 0us
Learning rate set to 0.039622
0:	learn: 0.6545572	total: 5.06ms	remaining: 5.06s
200:	learn: 0.2887860	total: 856ms	remaining: 3.4s
400:	learn: 0.2721197	total: 1.75s	remaining: 2.62s
600:	learn: 0.2627770	total: 2.55s	remaining: 1.69s
800:	learn: 0.2558051	total: 3.38s	remaining: 839ms
999:	learn: 0.2504169	total: 4.21s	

In [39]:
voting_model = VotingClassifier(
                    estimators=[
                        ('Ada_mode',Ada_model),
                        ('lgbm_model',lgbm_model),
                        ('cb_model',cb_model)
                    ],
                    voting='soft'
                )
# 교차검증 확인
result = cross_val_score(voting_model,
                          X_train,
                          y_train,
                          cv = 5)
result.mean()
# 학습
voting_model.fit(X_train,y_train)
# 예측
pre = voting_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('voting_model6.csv', index = False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5636, number of negative: 17808
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000749 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 340
[LightGBM] [Info] Number of data points in the train set: 23444, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
Learning rate set to 0.039622
0:	learn: 0.6535233	total: 5.24ms	remaining: 5.24s
200:	learn: 0.2866496	total: 906ms	remaining: 3.6s
400:	learn: 0.2692787	total: 1.72s	remaining: 2.56s
600:	learn: 0.2596979	total: 2.53s	remaining: 1.68s
800:	learn: 0.2531792	total: 3.35s	remaining: 832ms
999:	learn: 0.2480846	total: 4.16s	remaining: 0us
[LightGBM] [Warning] Found